In [1]:
import os
from py2neo import Graph
import genelab_utils as gl
import neo4j_utils

## Setup Environment Variables
To update the environment, update the "../.env" file.

In [2]:
# Node and relationship directory paths
node_dir, rel_dir = gl.setup_environment()

Environment setup for KG version: v0.0.3


### Connect to the local Neo4j Graph database

In [3]:
database = os.environ.get("NEO4J_DATABASE")
username = os.environ.get("NEO4J_USERNAME")
password = os.environ.get("NEO4J_PASSWORD")
stylesheet = os.environ.get("NEO4J_STYLESHEET")

graph = Graph("bolt://localhost:7687", name=database, user=username, password=password)

## Gene Methylation vs Downregulation
This query finds examples where a gene is differentially hypermethylated in the promoter region and downregulated for a Ground Control vs. Space Flight comparison.

In [4]:
query = """
MATCH p = (m:Mission)-->(s1:Study)-->(a1:Assay)-[m1]->(y:MethylationRegion)
           <--(g:MGene)<-[m2]-(a2:Assay)<--(s2:Study)
WHERE s1 = s2 // within the same study
  AND "Ground Control" IN a1.factors_1 // Ground Control vs. Space Flight comparisions
  AND "Space Flight" IN a1.factors_2
  AND "Ground Control" IN a2.factors_1
  AND "Space Flight" IN a2.factors_2
  AND y.in_promoter = true // Methylated in the promoter region
  AND m1.methylation_diff > 0 // positive difference in methylation
  AND m2.log2fc < 0 // downregulated gene
RETURN p
"""

subgraph1 = graph.run(query).to_subgraph()

In [8]:
# Rerun this cell again if the layout doesn't look good
# Use the mouse to drag and zoom, or move nodes
# Click on a node to display the node properties
widget1 = neo4j_utils.draw_graph(subgraph1, stylesheet)
widget1.layout.height = "1014px"
widget1.set_layout(name="cola", padding=00, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=15000)
widget1

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'padding': 0, 'nodeSpacing': 65, 'nodeDimensionsIncludeLabel…

Open a static image of thisgraph visualization for [OSD-105](../docs/osd-105_omics.png)

## Comparision of Gene Methylation vs. Gene Expression
Here we compare the change in methylation (methylation_diff in %) in the promoter region with the change the log2 fold change in expression (log2fc) for Ground Control vs. Space Flight samples in mice.

While there is no correlation, it's interesting to note that Fibroblast growth factor receptor 3 (FGFR3) is a top hit. It is among other functions involved in the regulation of osteogenesis and postnatal bone mineralization by osteoblasts (see: https://www.uniprot.org/uniprotkb/Q61851/entry). The promoter hypermethylation of FGFR3 may contribute to its transcriptional repression in spaceflown mice.

In [6]:
query = """
MATCH (s1:Study)-->(a1:Assay)-[m1]->(y:MethylationRegion)
      <--(g:MGene)<-[m2]-(a2:Assay)<--(s2:Study)
      
WHERE s1 = s2 // within the same study
  AND "Ground Control" IN a1.factors_1 // Ground Control
  AND "Space Flight" IN a1.factors_2   // vs. Space Flight
  AND "Ground Control" IN a2.factors_1
  AND "Space Flight" IN a2.factors_2
  AND y.in_promoter = true // Methylated in the promoter region
  
RETURN m1.methylation_diff AS methylation_diff, m2.log2fc AS log2fc, 
       g.name AS gene, s1.organism as organism, a1.material_name_1 AS anatomy, s1.identifier AS study
ORDER BY methylation_diff DESC;
"""
graph.run(query).to_data_frame()

,methylation_diff,log2fc,gene,organism,anatomy,study
0,17.701717,-0.643865,fibroblast growth factor receptor 3,Mus musculus,tibialis anterior,OSD-105
1,11.834080,0.190048,dystrophia myotonica-protein kinase,Mus musculus,tibialis anterior,OSD-105
2,6.466081,-0.342864,progestin and adipoQ receptor family member IX,Mus musculus,tibialis anterior,OSD-105
3,1.819836,0.265232,insulin degrading enzyme,Mus musculus,tibialis anterior,OSD-105
4,-1.691213,0.203576,NADH dehydrogenase subunit 2,Mus musculus,tibialis anterior,OSD-105
5,-2.594896,-0.286396,"phosphate cytidylyltransferase 2, ethanolamine",Mus musculus,tibialis anterior,OSD-105
6,-4.385965,-0.577865,docking protein 5,Mus musculus,tibialis anterior,OSD-105
7,-6.217617,-0.152480,2-aminoethanethiol (cysteamine) dioxygenase,Mus musculus,tibialis anterior,OSD-105
8,-7.277940,0.403671,"H19, imprinted maternally expressed transcript",Mus musculus,tibialis anterior,OSD-105
9,-14.498471,1.354894,transmembrane protein 254,Mus musculus,tibialis anterior,OSD-105


In [7]:
# Get some metadata about this study
query = """
MATCH (m:Mission)-->(s:Study)
WHERE s.identifier = 'OSD-105'
RETURN s.identifier AS study, s.project_title AS title, m.name AS mission, 
       m.flight_program AS progam, m.start_date AS start_date, m.end_date AS end_date;
"""
graph.run(query).to_data_frame()

,study,title,mission,progam,start_date,end_date
0,OSD-105,Rodent Research 1,SpaceX-4,International Space Station (ISS),2014-09-21,2014-10-25
